# Problemas resueltos — Capítulo 4

En este notebook se resuelven los ejercicios 4.3.1, 4.3.2, 4.4.1 y 4.4.2. Cada celda contiene el enunciado breve, el código y una interpretación corta del resultado.

Si tu paquete `quantum_simulator` está disponible en el `PYTHONPATH`, el notebook importará las clases desde allí; en caso contrario se usan implementaciones alternativas incluidas para que todo sea ejecutable de forma independiente.

## Celda 1 — Importaciones y clases fallback

Intentamos importar las clases desde el paquete del proyecto. Si la importación falla, definimos implementaciones equivalentes mínimas para `QuantumSystem`, `Observable` y `QuantumDynamics`.

In [ ]:
# Celda 1: Importaciones y fallback
import sys
sys.path.append('..')

import numpy as np

try:
    from quantum_simulator.core import QuantumSystem
    from quantum_simulator.observables import Observable
    from quantum_simulator.dynamics import QuantumDynamics
    print("Importadas clases desde quantum_simulator.*")
except Exception as e:
    print("No se pudo importar desde quantum_simulator. Usando implementaciones locales. Error:", e)
    # Implementación mínima de QuantumSystem
    class QuantumSystem:
        def __init__(self, N):
            self.N = int(N)
            self._state = np.zeros(self.N, dtype=complex)
            self._state[0] = 1.0
        def set_state(self, amplitudes):
            a = np.asarray(amplitudes, dtype=complex).reshape(-1)
            if a.size != self.N:
                raise ValueError("Longitud de amplitudes distinta a N")
            norm = np.linalg.norm(a)
            if norm == 0:
                raise ValueError("El estado no puede ser cero")
            self._state = a / norm
            return self._state
        @property
        def state(self):
            return self._state.copy()
        def position_probability(self, i):
            return float(np.abs(self._state[i])**2)
        def transition_probability(self, other_ket):
            b = np.asarray(other_ket, dtype=complex).reshape(-1)
            if b.size != self.N:
                raise ValueError("Longitud de ket objetivo distinta a N")
            b = b / np.linalg.norm(b)
            amp = np.vdot(b, self._state)
            return float(np.abs(amp)**2)

    # Implementación mínima de Observable
    class Observable:
        def __init__(self, matrix):
            self.M = np.asarray(matrix, dtype=complex)
            if self.M.shape[0] != self.M.shape[1]:
                raise ValueError("Observable debe ser matriz cuadrada")
        def mean(self, state):
            psi = np.asarray(state, dtype=complex).reshape(-1)
            psi = psi / np.linalg.norm(psi)
            return float(np.vdot(psi, self.M.dot(psi)))
        def variance(self, state):
            psi = np.asarray(state, dtype=complex).reshape(-1)
            psi = psi / np.linalg.norm(psi)
            exp = np.vdot(psi, self.M.dot(psi))
            exp2 = np.vdot(psi, self.M.dot(self.M).dot(psi))
            return float(exp2 - exp**2)
        def eigenvalues_eigenvectors(self):
            vals, vecs = np.linalg.eig(self.M)
            return vals, vecs
        def probability_eigenvalue(self, state, index):
            vals, vecs = np.linalg.eig(self.M)
            psi = np.asarray(state, dtype=complex).reshape(-1)
            psi = psi / np.linalg.norm(psi)
            v = vecs[:, index]
            v = v / np.linalg.norm(v)
            return float(np.abs(np.vdot(v.conj(), psi))**2)

    # Implementación mínima de QuantumDynamics (no usada aquí, pero incluida)
    class QuantumDynamics:
        def __init__(self, initial_state):
            a = np.asarray(initial_state, dtype=complex).reshape(-1)
            self.state = a / np.linalg.norm(a)
        def apply_unitary_evolution(self, unitary_list):
            st = self.state.copy()
            for U in unitary_list:
                st = np.asarray(U, dtype=complex).dot(st)
            self.state = st
            return self.state

## Celda 2 — Problema 4.3.1

**Enunciado (breve):** Dado un sistema de 3 posiciones con un estado inicial, calcular la probabilidad de encontrar la partícula en cada posición.

In [ ]:
# Celda 2: Problema 4.3.1
print("=== Problema 4.3.1 ===")
system_431 = QuantumSystem(3)
system_431.set_state([1+1j, 0, 1j])

print("Probabilidades:")
for i in range(3):
    prob = system_431.position_probability(i)
    print(f"Posición {i}: {prob:.4f}")

## Celda 3 — Problema 4.3.2

**Enunciado (breve):** Calcular la probabilidad de transición desde el estado actual del sistema hacia el ket objetivo `target_ket`.

In [ ]:
# Celda 3: Problema 4.3.2
print("\n=== Problema 4.3.2 ===")
target_ket = [1j, 1, 1+1j]
transition_prob = system_431.transition_probability(target_ket)
print(f"Probabilidad de transición: {transition_prob:.4f}")

## Celda 4 — Problema 4.4.1

**Enunciado (breve):** Dado un observable (matriz), calcular la media y la varianza sobre el estado `system_431.state`. También verificar hermiticidad si lo deseas.

In [ ]:
# Celda 4: Problema 4.4.1
print("\n=== Problema 4.4.1 ===")
observable_matrix = [[1, 0, 1j], [0, 1, 0], [-1j, 0, 1]]
observable = Observable(observable_matrix)

mean_val = observable.mean(system_431.state)
variance_val = observable.variance(system_431.state)

print(f"Media del observable: {mean_val:.4f}")
print(f"Varianza del observable: {variance_val:.4f}")

## Celda 5 — Problema 4.4.2

**Enunciado (breve):** Obtener valores y vectores propios del observable y calcular la probabilidad de colapsar a cada uno de ellos desde el estado `system_431.state`. 

In [ ]:
# Celda 5: Problema 4.4.2
print("\n=== Problema 4.4.2 ===")
eigenvalues, eigenvectors = observable.eigenvalues_eigenvectors()
print("Valores propios:", eigenvalues)

for i in range(len(eigenvalues)):
    prob = observable.probability_eigenvalue(system_431.state, i)
    print(f"P(colapsar a vector propio {i}): {prob:.4f}")

## Interpretación final

- Las probabilidades deben sumar aproximadamente 1 (salvo redondeo). 
- La media y la varianza ofrecen medida estadística del observable sobre el estado dado.
- Los valores propios corresponden a los posibles resultados de la medida y las probabilidades calculadas son las probabilidades de obtener cada resultado.